In [1]:
import numpy as np
import pandas as pd

In [2]:
NPI_COLS = ['C1_School closing', 'C2_Workplace closing',
           'C3_Cancel public events', 'C4_Restrictions on gatherings',
           'C5_Close public transport', 'C6_Stay at home requirements',
           'C7_Restrictions on internal movement',
           'C8_International travel controls', 'E1_Income support',
           'E2_Debt/contract relief', 'H1_Public information campaigns',
           'H2_Testing policy', 'H3_Contact tracing', 'H6_Facial Coverings', 
           'H7_Vaccination policy','H8_Protection of elderly people']

WINDOW = 7

# Population

https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/bulletins/annualmidyearpopulationestimates/mid2020#:~:text=1.,%25)%20since%20mid%2Dyear%202019.

In [3]:
pop_uk = pd.DataFrame({'CountryName': ['Wales', 'England', 'Scotland', 'Northern Ireland'],
                       'Population2020': [3170000, 56550000, 5466000, 1896000]})

# Read Data
pop = pd.read_csv('../Data/Population.csv')

# Dropping Columns
pop.drop(columns=['Country Code'], inplace=True)

# Renaming COlumns
pop.columns = ['CountryName', 'Population2020']

# Drop missing Values
pop.dropna(inplace=True)

# add UK Regions
pop = pd.concat([pop, pop_uk], axis='rows').reset_index(drop=True)

# Russia & Faroe Islands
pop.CountryName.replace(to_replace='Russian Federation',  value='Russia', inplace=True)
pop.CountryName.replace(to_replace='Faroe Islands',  value='Faeroe Islands', inplace=True)

# Change the type of Popluation column
pop['Population2020'] = pop['Population2020'].astype('int')

pop

,CountryName,Population2020
0,Aruba,106766
1,Africa Eastern and Southern,677243299
2,Afghanistan,38928341
3,Africa Western and Central,458803476
4,Angola,32866268
...,...,...
263,Zimbabwe,14862927
264,Wales,3170000
265,England,56550000
266,Scotland,5466000


# Vaccination

In [4]:
# Read
vaccine = pd.read_csv('../Data/Vaccination.csv', index_col=0, parse_dates=['date'])

# Drop Columns
vaccine.drop(columns=['iso_code'], inplace=True)

# Renaming Columns
vaccine.columns = ['CountryName', 'Date', 'Vaccinated', 'FullyVaccinated', 'DailyVaccination']

# daily vaccination: put 0 for missing values
vaccine.DailyVaccination.fillna(0, inplace=True)

# FullyVaccinated: forward fill (fill with the latest value, since it is cumulative), and 0 for the rest
vaccine.update(vaccine.groupby('CountryName')['FullyVaccinated'].ffill().fillna(0))

# Vaccinated: first forward fill, then replace with 0 for the rest.
vaccine.update(vaccine.groupby('CountryName')['Vaccinated'].ffill().fillna(0))

# Countries
vaccine.CountryName.replace(to_replace='Czechia',  value='Czech Republic', inplace=True)
vaccine.CountryName.replace(to_replace='Slovakia',  value='Slovak Republic', inplace=True)

vaccine

,CountryName,Date,Vaccinated,FullyVaccinated,DailyVaccination
0,Afghanistan,2021-02-22,0.0,0.0,0.0
1,Afghanistan,2021-02-23,0.0,0.0,1367.0
2,Afghanistan,2021-02-24,0.0,0.0,1367.0
3,Afghanistan,2021-02-25,0.0,0.0,1367.0
4,Afghanistan,2021-02-26,0.0,0.0,1367.0
...,...,...,...,...,...
33202,Zimbabwe,2021-07-09,875182.0,588883.0,15907.0
33203,Zimbabwe,2021-07-10,875182.0,588883.0,16878.0
33204,Zimbabwe,2021-07-11,895980.0,595417.0,18598.0
33205,Zimbabwe,2021-07-12,926312.0,605556.0,23205.0


# Weather 

In [5]:
weather = pd.read_csv('../Data/Weather.csv', index_col=0, parse_dates=['date_time'])
weather.columns = ['CountryName', 'Date', 'HeatIndexC', 'humidity', 
                  'tempC', 'windspeedKmph', 'precipMM', 'DewPointC', 'pressure']

weather

,CountryName,Date,HeatIndexC,humidity,tempC,windspeedKmph,precipMM,DewPointC,pressure
0,Afghanistan,2020-01-01,1,46,3,9,0.4,-10,1026
1,Afghanistan,2020-01-02,-2,95,-1,5,6.5,-3,1028
2,Afghanistan,2020-01-03,-4,87,-1,5,0.9,-5,1026
3,Afghanistan,2020-01-04,0,42,4,6,0.0,-12,1023
4,Afghanistan,2020-01-05,-2,52,2,5,2.1,-11,1022
...,...,...,...,...,...,...,...,...,...
552,Zimbabwe,2021-07-06,16,50,21,13,0.0,3,1023
553,Zimbabwe,2021-07-07,15,64,22,10,0.1,6,1023
554,Zimbabwe,2021-07-08,16,52,23,7,0.0,3,1024
555,Zimbabwe,2021-07-09,18,40,24,7,0.0,0,1023


# Covid

In [6]:
covid = pd.read_csv('../Data/Covid19_Europe_20210710.csv', 
                    index_col=0, 
                    parse_dates=['Date'])

# Filter Countries
covid = covid[['CountryName', 'RegionName', 'Date', 'ConfirmedCases', 'ConfirmedDeaths'] + NPI_COLS]

# Considering the Regions of UK as separate countries
covid = covid.loc[~((covid.CountryName == 'United Kingdom') & (covid.RegionName.isna()))]

is_england = ((covid.CountryName == 'United Kingdom') & (covid.RegionName == 'England')).values
is_ireland = ((covid.CountryName == 'United Kingdom') & (covid.RegionName == 'Northern Ireland')).values
is_scotland = ((covid.CountryName == 'United Kingdom') & (covid.RegionName == 'Scotland')).values
is_wales = ((covid.CountryName == 'United Kingdom') & (covid.RegionName == 'Wales')).values

covid.loc[is_england, 'CountryName'] = 'England'
covid.loc[is_ireland, 'CountryName'] = 'Northern Ireland'
covid.loc[is_scotland, 'CountryName'] = 'Scotland'
covid.loc[is_wales, 'CountryName'] = 'Wales'

# Removing Region Name
covid.drop(columns=['RegionName'], inplace=True)

# Interpolation & Drop country which no number of cases is available
covid['ConfirmedCases'] = covid.groupby('CountryName')['ConfirmedCases'].\
                                apply(lambda group: group.interpolate(limit_area='inside').ffill().fillna(0))

# Interpolation & Drop country which no number of deaths is available
covid['ConfirmedDeaths'] = covid.groupby('CountryName')['ConfirmedDeaths'].\
                                apply(lambda group: group.interpolate(limit_area='inside').ffill().fillna(0))


# Fill missing values for Policies
for npi_column in NPI_COLS:
    covid[npi_column] = covid.groupby('CountryName')[npi_column].ffill().fillna(0).astype('int')

# Merging

In [7]:
# Merge covid and population
final_df = pd.merge(covid, pop, how='left', on='CountryName')

# Compute number of new cases and deaths each day
final_df['NewCases'] = final_df.groupby('CountryName').ConfirmedCases.diff().fillna(0)
final_df['NewDeaths'] = final_df.groupby('CountryName').ConfirmedDeaths.diff().fillna(0)

# Replace negative values (which do not make sense for these columns) with 0
final_df['NewCases'] = final_df['NewCases'].clip(lower=0)
final_df['NewDeaths'] = final_df['NewDeaths'].clip(lower=0)

# Compute smoothed versions of new cases and deaths each day
final_df['SmoothNewCases'] = final_df.groupby('CountryName')['NewCases'].rolling(
    window = WINDOW, center=False).mean().fillna(0).reset_index(0, drop=True).round()
final_df['SmoothNewDeaths'] = final_df.groupby('CountryName')['NewDeaths'].rolling(
    window = WINDOW, center=False).mean().fillna(0).reset_index(0, drop=True).round()

# Compute percent change in new cases and deaths each day
final_df['CaseRatio'] = final_df.groupby('CountryName').SmoothNewCases.pct_change().fillna(0).replace(np.inf, 0) + 1
final_df['DeathRatio'] = final_df.groupby('CountryName').SmoothNewDeaths.pct_change().fillna(0).replace(np.inf, 0) + 1

# Add column for proportion of population infected
final_df['ProportionInfected'] = final_df['ConfirmedCases'] / final_df['Population2020']

# Create column of value to predict
final_df['PredictionRatio'] = final_df['CaseRatio'] / (1 - final_df['ProportionInfected'])

# Merge with vaccine
final_df = pd.merge(final_df, vaccine, how='left', on=['CountryName', 'Date'])
      
# Merge with weather
final_df = pd.merge(final_df, weather, how='left', on=['CountryName', 'Date'])

final_df.ConfirmedCases = final_df.ConfirmedCases.astype('int')
final_df.ConfirmedDeaths = final_df.ConfirmedDeaths.astype('int')

# View

In [9]:
final_df

,CountryName,Date,ConfirmedCases,ConfirmedDeaths,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,...,Vaccinated,FullyVaccinated,DailyVaccination,HeatIndexC,humidity,tempC,windspeedKmph,precipMM,DewPointC,pressure
0,Albania,2020-01-01,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,6.0,51.0,11.0,6.0,0.0,-3.0,1027.0
1,Albania,2020-01-02,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,6.0,47.0,11.0,6.0,0.0,-5.0,1031.0
2,Albania,2020-01-03,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,6.0,47.0,10.0,4.0,0.0,-5.0,1027.0
3,Albania,2020-01-04,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,8.0,50.0,11.0,3.0,0.0,-2.0,1021.0
4,Albania,2020-01-05,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,5.0,55.0,8.0,14.0,0.0,-3.0,1021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28402,Ukraine,2021-07-06,2306006,54815,3,2,1,3,0,0,...,2136561.0,955607.0,72671.0,23.0,83.0,25.0,17.0,3.1,19.0,1011.0
28403,Ukraine,2021-07-07,2307068,54860,3,2,1,3,0,0,...,2178590.0,1018258.0,74472.0,22.0,72.0,26.0,15.0,0.0,16.0,1020.0
28404,Ukraine,2021-07-08,2308142,54894,3,2,1,3,0,0,...,2223406.0,1082602.0,76914.0,24.0,63.0,28.0,13.0,0.0,16.0,1025.0
28405,Ukraine,2021-07-09,2309264,54917,3,2,1,3,0,0,...,2277226.0,1143442.0,83765.0,25.0,59.0,29.0,9.0,0.0,16.0,1024.0


# Save 

In [8]:
final_df.to_csv('../Data/Covid19_Europe_20210710_preprocessed.csv')